In [62]:
#one-hot encoding
#use all data of day as one vector
#more layers

import pandas as pd
import csv

df = pd.read_csv('raw/삼성전자.csv').drop(['Date','Adj Close'], axis=1)
iter_len = len(df['Close'])
filename = 'prep1/삼성전자.csv'


# Labeling: increase = 1, decrease = -1

with open(filename, 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow("Open,High,Low,Close,Volume".split(",")+['Label'])
    for i in range(iter_len-1):
        line1 = df.iloc[i]
        line2 = df.loc[i+1]
        label = 1 if line2['Close']>line1['Close'] else -1
        line1['Label']=label
        wt.writerow(line1.tolist())

<ipython-input-62-c1b2af8ef39f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  line1['Label']=label


In [63]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


f = pd.read_csv(filename)
labels = f.axes[1]
rawX = f.drop(columns=labels[-1], axis=1).values
rawY = f.drop(columns=labels[:-1], axis=1).values

In [64]:
newY = []

for val in rawY:
    if val == 1:
        newY.append(np.array([1,0]))
    else: newY.append(np.array([0,1]))


x=np.array(rawX)
y=np.array(newY)

if torch.cuda.is_available():
    x=torch.tensor(x).cuda()
    y=torch.tensor(y).cuda()
    print("CUDA available")
else:
    x=torch.tensor(x)
    y=torch.tensor(y)

y

tensor([[0, 1],
        [1, 0],
        [0, 1],
        ...,
        [0, 1],
        [1, 0],
        [0, 1]], dtype=torch.int32)

In [65]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.l1 = nn.Linear(D_in, H)
        self.h1 = nn.Linear(H, 2*H)
        self.h2 = nn.Linear(2*H, 4*H)
        self.l2 = nn.Linear(4*H, D_out)

    def forward(self, x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.h1(x))
        x = F.leaky_relu(self.h2(x))
        y = F.softmax(self.l2(x))
        return y


D_in, D_out = 5, 2 #input, output 

def run(H, lr, epochs):
  
    #training
    
    model = Net(D_in, H, D_out)
    if torch.cuda.is_available(): model = model.cuda()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        y_pred = model(x.float())
        loss = criterion(y_pred, y.float())
        print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  ###############################################################################
  
    #evaluation
    total=0
    correct=0
    
    for idx, inp in enumerate(x):
        total+=1
        y_pred = list(model(inp.float()).cpu().detach().numpy())
        y_truth = list(newY[idx])
        print(y_pred, y_truth)
        if y_pred.index(max(y_pred))==y_truth.index(max(y_truth)): correct+=1
    print(round(100*correct/total,2))


In [66]:
run(24, 1e-3, 1000)

<ipython-input-65-bc91a80cdad3>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.l2(x))


0 5300.0
1 5300.0
2 5300.0
3 5300.0
4 5300.0
5 5300.0
6 5300.0
7 5300.0
8 5300.0
9 5300.0
10 5300.0
11 5300.0
12 5300.0
13 5300.0
14 5300.0
15 5300.0
16 5300.0
17 5300.0
18 5300.0
19 5300.0
20 5300.0
21 5300.0
22 5300.0
23 5300.0
24 5300.0
25 5300.0
26 5300.0
27 5300.0
28 5300.0
29 5300.0
30 5300.0
31 5300.0
32 5300.0
33 5300.0
34 5300.0
35 5300.0
36 5300.0
37 5300.0
38 5300.0
39 5300.0
40 5300.0
41 5300.0
42 5300.0
43 5300.0
44 5300.0
45 5300.0
46 5300.0
47 5300.0
48 5300.0
49 5300.0
50 5300.0
51 5300.0
52 5300.0
53 5300.0
54 5300.0
55 5300.0
56 5300.0
57 5300.0
58 5300.0
59 5300.0
60 5300.0
61 5300.0
62 5300.0
63 5300.0
64 5300.0
65 5300.0
66 5300.0
67 5300.0
68 5300.0
69 5300.0
70 5300.0
71 5300.0
72 5300.0
73 5300.0
74 5300.0
75 5300.0
76 5300.0
77 5300.0
78 5300.0
79 5300.0
80 5300.0
81 5300.0
82 5300.0
83 5300.0
84 5300.0
85 5300.0
86 5300.0
87 5300.0
88 5300.0
89 5300.0
90 5300.0
91 5300.0
92 5300.0
93 5300.0
94 5300.0
95 5300.0
96 5300.0
97 5300.0
98 5300.0
99 5300.0
100 5300.0

767 5300.0
768 5300.0
769 5300.0
770 5300.0
771 5300.0
772 5300.0
773 5300.0
774 5300.0
775 5300.0
776 5300.0
777 5300.0
778 5300.0
779 5300.0
780 5300.0
781 5300.0
782 5300.0
783 5300.0
784 5300.0
785 5300.0
786 5300.0
787 5300.0
788 5300.0
789 5300.0
790 5300.0
791 5300.0
792 5300.0
793 5300.0
794 5300.0
795 5300.0
796 5300.0
797 5300.0
798 5300.0
799 5300.0
800 5300.0
801 5300.0
802 5300.0
803 5300.0
804 5300.0
805 5300.0
806 5300.0
807 5300.0
808 5300.0
809 5300.0
810 5300.0
811 5300.0
812 5300.0
813 5300.0
814 5300.0
815 5300.0
816 5300.0
817 5300.0
818 5300.0
819 5300.0
820 5300.0
821 5300.0
822 5300.0
823 5300.0
824 5300.0
825 5300.0
826 5300.0
827 5300.0
828 5300.0
829 5300.0
830 5300.0
831 5300.0
832 5300.0
833 5300.0
834 5300.0
835 5300.0
836 5300.0
837 5300.0
838 5300.0
839 5300.0
840 5300.0
841 5300.0
842 5300.0
843 5300.0
844 5300.0
845 5300.0
846 5300.0
847 5300.0
848 5300.0
849 5300.0
850 5300.0
851 5300.0
852 5300.0
853 5300.0
854 5300.0
855 5300.0
856 5300.0
857 5300.0

[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[1.0, 8.66042e-40] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[1.0, 4.368559e-39] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] 

[0.0, 1.0] [0, 1]
[1.0, 8e-45] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[1.0, 0.0] [0, 1]
[1.0, 0.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[1.0, 0.0] [0, 1]
[1.0, 0.0] [0, 1]
[1.0, 0.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[1.0, 0.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.

...

[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0]

[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[1.0, 0.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0]

[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0]

[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0]

[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [0, 1]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [1, 0]
[0.0, 1.0] [0, 1]
[0.0, 1.0]